In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project
%cd /content/drive/MyDrive/Colab\ Notebooks/italian_teacher

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/italian_teacher


In [2]:
# Install dependencies
!pip install -q transformers trl accelerate peft datasets spacy sentence-transformers bitsandbytes
!python -m spacy download it_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 123.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:


import json
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from datasets import Dataset
from getpass import getpass
from pathlib import Path

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")




secrets_paths = [
    Path.home() / "Google Drive" / "My Drive" / ".secrets.json",  # Local path
    Path("/content/drive/My Drive/.secrets.json"),                # Colab path
    Path('.secrets.json')                                         # Current directory
]

for path in secrets_paths:
    if path.exists():
        print(f"✅ Loading API keys from {path}")
        with open(path, 'r') as f:
            secrets = json.load(f)

        loaded_keys = []
        for key, value in secrets.items():
            if value and value not in ["your-openai-key-here", "your-google-key-here", ""]:
                os.environ[key] = value
                loaded_keys.append(key)

        if loaded_keys:
            print(f"   Loaded {len(loaded_keys)} API key(s)")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA L4
✅ Loading API keys from /content/drive/My Drive/.secrets.json
   Loaded 9 API key(s)


In [6]:
!pip install json5

In [12]:
from src.rl.reward_function import ExerciseRewardFunction
!pip install anthropic groq

# Initialize reward function (loads vocabulary + models)
print("Loading reward function...")
reward_fn = ExerciseRewardFunction(disabled_scorers=[])
print("✅ Reward function ready")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 16.2 MB/s eta 0:00:00
Loading reward function...
Loading spaCy model: it_core_news_sm...
✅ spaCy model loaded
Reward function will use device: cuda
     ✅ Gemini: 4 API key(s)
     ✅ OpenAI: configured
     ✅ Anthropic: configured
     ✅ Groq: configured
     ✅ DeepSeek: configured
     ✅ Cerebras: configured
  ✅ LLM API Handler initialized
     Providers: gemini, openai, anthropic, groq, deepseek, cerebras
     Total models: 12
Initializing scorers...
  ✅ LLM scoring enabled for cefr_alignment (batch size: 10)
  ✅ LLM scoring enabled for fluency (batch size: 10)
  ✅ LLM scoring enabled for grammar_correctness (batch size: 10)
  ✅ LLM scoring enabled for coherence (batch size: 10)
Loading sentence transformer for topic similarity...
✅ Sentence transformer loaded in cuda
  ✅ LLM topic checking enabled (OpenAI API)
✅ Reward function initialized. Acti

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your Round 1 GRPO-trained model for evaluation
MODEL_PATH = "./models/TeacherPet_italian_grpo"  # ← Round 1 GRPO model (Oct 17)

print(f"Loading model from: {MODEL_PATH}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    load_in_8bit=True
)
model.eval()

print(f"✅ Model loaded on device: {next(model.parameters()).device}")

Loading model from: ./models/TeacherPet_italian_grpo


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded on device: cuda:0


In [8]:
import os

# Load pre-generated validation requests
if os.path.exists("src/rl/validation_requestss.json"):
    print("Loading existing validation requests...")
    with open("src/rl/validation_requestss.json", "r") as f:
        validation_requests = json.load(f)
else:
    # If not exists, generate them
    from src.rl.generate_training_requests import generate_training_requests
    print("Generating new validation requests...")
    validation_requests = generate_training_requests(
        num_requests=20,
        output_path="src/rl/validation_requests.json"
    )

print(f"✅ Loaded {len(validation_requests)} validation requests")

Generating new validation requests...
Loading vocabulary...
✅ Loaded 16887 Italian words from vocabulary list
Generating 20 training requests...
✅ Saved 20 requests to src/rl/validation_requests.json

📊 Request Statistics:
By CEFR level:
  A1: 1
  A2: 3
  B1: 6
  B2: 7
  C1: 1
  C2: 2

Top 5 grammar focuses:
  verb_conjugation: 3
  verbi_riflessivi: 2
  past_tense: 2
  imperfect_tense: 2
  trapassato_prossimo: 2
✅ Loaded 20 validation requests


In [13]:
from src.rl.iterative_training import evaluate_model_on_requests, create_round2_dataset
# After Round 1 training:
low, high = await evaluate_model_on_requests(model, tokenizer, reward_fn, validation_requests, batch_size=50, output_path='eval.json')


✅ Model is already quantized and mapped to devices.
✅ SentenceTransformer moved to CUDA for topic scoring.
📊 Evaluating 20 requests in batches of 50 (max_new_tokens=400)
Model device: cuda:0


Evaluating Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



✅ Evaluation complete. Processed: 20
  Low scoring (<92): 20
  High scoring (>=92): 0
  Avg score (processed): 74.01
💾 Final results saved to eval.json
